In [64]:
import requests
import xmltodict
import urllib, urllib.request
import mysql.connector
import time
from IPython.display import clear_output
import uuid

In [65]:
headers = {"x-api-key": "M7HSjQNeTfai6l7JUiDZB8XYc85BHnHt3R0NXSEd"}
# Establish database connection
cnx = mysql.connector.connect(user='david', password='daviddung1993',
                              host='127.0.0.1',
                              database='computervision')
cursor = cnx.cursor()

# Check if there are duplicate entries

In [85]:
cursor.execute("select p.Title, count(*) from Papers p group by p.Title having count(*) > 1")
duplicate_papers = cursor.fetchall()
duplicate_papers

[('Medical Image Registration', 2),
 ('Progress in Pattern Recognition, Image Analysis, Computer Vision, and Applications',
  2),
 ('Optical Character Recognition for Cursive Handwriting', 2),
 ('Distance errors correction for the time of flight (ToF) cameras', 2),
 ('An Overview of Steganography', 2),
 ('Counting people in crowd open scene based on grey level dependence matrix',
  2),
 ('Study and comparison of various image edge detection techniques', 2),
 ('Hadamard Matrices and Their Applications', 2),
 ('Visual vibrometry: Estimating material properties from small motions in video',
  2),
 ('A parallel design and implementation for backpropagation neural network using MIMD architecture',
  2),
 ('An Introduction to Multivariate Statistical Analysis', 2),
 ('Rich intrinsic image decomposition of outdoor scenes from multiple views',
  2),
 ('Closed-Form Solution of Absolute Orientation Using Orthonormal Matrices',
  2),
 ('Digital Color Imaging Handbook', 2),
 ('Pseudo-random Graphs

In [89]:
title = "Medical Image Registration"
cursor.execute("select p.PaperID, p.Leaf, p.ReferenceCount from Papers p where p.Title = %s group by p.PaperID, p.Leaf", [title])
informations = cursor.fetchall()

print(title)
for information in informations:
    print(information[0])
    cursor.execute("select b2.AuthoredByID, a.Name, count(*) from Papers p, authoredBy b, Authors a, authoredBy b2 where p.PaperID=%s and p.PaperID = b.PaperID and b.AuthoredByID = a.AuthorID and a.AuthorID = b2.AuthoredByID group by a.AuthorID, a.Name",[information[0]])
    author_information = cursor.fetchall()
    print(author_information)
    print(f"Is Leaf: {information[1]}")
    print(f"ReferenceCount: {information[2]}")
    cursor.execute("select count(*) from referencedBy r where r.ReferenceID=%s",[information[0]])
    referencedBy = cursor.fetchall()
    print(f"Its referenced by {referencedBy}")
    cursor.execute("select count(*) from referencedBy r where r.ReferencedByID=%s",[information[0]])
    references = cursor.fetchall()
    print(f"It references {references}")
    print("----------------------------")

Medical Image Registration
00506b5142f13000a8849273f1aca9f33f39c38c
[('121013794', 'M. Shubhakanta Singh', 1), ('2080397101', 'Balraj Naren', 1)]
Is Leaf: 1
ReferenceCount: 0
Its referenced by [(4,)]
It references [(0,)]
----------------------------
47c880403441c9245b6cf713f7925ddde17fb53a
[('143733284', 'Mark Holden', 6), ('144159315', 'Derek L Hill', 24), ('1739885', 'David Hawkes', 32), ('2672693', 'Philip G Batchelor', 7)]
Is Leaf: 1
ReferenceCount: 174
Its referenced by [(1,)]
It references [(0,)]
----------------------------


In [78]:
master_node = "003d82c5051e959d834de7cdda6ee8441fa0119a"
replace_node = "e429d550cac5a7d9f6941e3ff50cd1d9cc399ba4"

In [79]:
# Delete Duplicate referencedBy entries
cursor.execute("select * from referencedBy r where r.ReferenceID = %s or r.ReferenceID = %s", [master_node, replace_node])
list_referenced = cursor.fetchall()
list_referenced = [list_referenced[1] for list_referenced in list_referenced]
unique_duplicates = set([number for number in list_referenced if list_referenced.count(number) > 1])
for x in unique_duplicates:
    cursor.execute("DELETE FROM referencedBy r where r.ReferenceID = %s and r.ReferencedByID = %s", [replace_node, x])

# Delete Duplicate referenced entries
cursor.execute("select * from referencedBy r where r.ReferencedByID = %s or r.ReferencedByID = %s", [master_node, replace_node])
list_referenced = cursor.fetchall()
list_referenced = [list_referenced[1] for list_referenced in list_referenced]
unique_duplicates = set([number for number in list_referenced if list_referenced.count(number) > 1])
for x in unique_duplicates:
    cursor.execute("DELETE FROM referencedBy r where r.ReferencedByID = %s and r.ReferenceID = %s", [replace_node, x])

cursor.execute("UPDATE referencedBy SET ReferenceID = %s where ReferenceID = %s",[master_node, replace_node])
cursor.execute("UPDATE referencedBy SET ReferencedByID = %s where ReferencedByID = %s",[master_node, replace_node])

In [71]:
replace_node

'ea339bfdad0ba956f0bc28451d13f1ed534d2344'

In [82]:
# Delete Paper information
cursor.execute("DELETE FROM authoredBy b where b.PaperID=%s", [replace_node])
cursor.execute("DELETE FROM Papers p where p.PaperID=%s", [replace_node])

In [73]:
# Delete Author Information
replace_author = [('1583081280', 'LuCewu', 1), ('1643932411', 'JiaJiaya', 1), ('2094729437', 'XuLi', 1), ('91195604', 'XuYi', 1)]

for author in replace_author:
    cursor.execute("delete from Authors a where a.AuthorID=%s", [author[0]])

In [83]:
cnx.commit()

In [63]:
cursor.close()

True

In [33]:
cnx.close()